In [19]:
import pandas
import itertools as it
import string
from collections import Counter
import json

In [20]:
punctuation_translator = str.maketrans(
            string.punctuation, " " * len(string.punctuation))

In [21]:
with open('data/stopwords.txt') as fh:
    stopwords = set(fh.read().splitlines())

In [22]:
def normalise(text):
    return " ".join(
        text.translate(punctuation_translator).replace("\n", " ").lower().split()
    )

In [23]:
def tokenise(text):
    return [t.strip() for t in text.strip().split(" ")]

In [24]:
def parse_treatment_definitons(definion_file):
    treatment_set = set()
    treatment_mapping = {}
    max_length = 1
    for line in definion_file:
        line = line.strip()
        treatments = line.split(',')
        name = treatments[0]
        for treatment in treatments:
            treatment = tuple(tokenise(normalise(treatment)))
            max_length = max(len(treatment), max_length)
            treatment_set.add(treatment)
            treatment_mapping[treatment] = name
    return treatment_set, treatment_mapping, max_length

In [25]:
def window_sliding(iterable, n):
    gens = (
        it.chain(it.repeat(None, n - 1 - i), iterable, it.repeat(None, i))
        for i, gen in enumerate(it.tee(iterable, n)))
    return zip(*gens)

def find_treatments(text):
    tokens = tokenise(normalise(text))
    found_treatments = []
    for x in range(max_treatment_length, 0, -1):
        for window in window_sliding(tokens, x):
            if tuple(window) in treatment_set:
                found_treatments.append(treatment_mapping[tuple(window)])
    return found_treatments or None

In [26]:
def filter_stopwords(text):
    tokens = tokenise(normalise(text))
    return [t for t in tokens if len(t) > 2 and t not in stopwords]

In [27]:
with open('merged_tweets.jsonl', 'r') as fh:
    df = pandas.read_json(fh.read(), lines=True, convert_dates=True)

In [28]:
df = df.loc[(df['retweeted_status'].isnull()) & (df['lang'] == "en")]

In [29]:
df = df.loc[(df.created_at.dt.weekofyear > 48) | (df.created_at.dt.weekofyear < 3)]

In [30]:
df["extended_tweet"] = df["extended_tweet"].apply(lambda x: x["full_text"]  if type(x) == dict else x )

In [31]:
df['text'] = df['extended_tweet'].combine_first(df["text"])

In [32]:
df["text"] = df["text"].apply(lambda x: x.replace("\n", " "))

In [33]:
df["user_name"] = df["user"].apply(lambda x: x["screen_name"])

In [34]:
df = df[["id", "created_at", "text", "retweet_count", "user_name"]]

In [35]:
df = df.sort_values("created_at")

In [37]:
treatment_set, treatment_mapping, max_treatment_length = parse_treatment_definitons(open("data/treatment_definitons.txt", 'r'))

In [38]:
df['treatments'] = df['text'].apply(find_treatments)

In [39]:
df['filtered_tokens'] = df['text'].apply(filter_stopwords)

In [40]:
df = df.loc[df['treatments'].notnull()]

In [41]:
df.to_csv("out.csv")

In [65]:
treatment_counts = Counter()
word_counts = Counter()
for row in df.itertuples():
    treatment_counts += Counter(row[5])
    word_counts += Counter(row[6])